In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Create images with white backgrounds
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fid = drive.ListFile({'q':"title='dataset.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('dataset.zip')

In [0]:
f.keys()

dict_keys(['id', 'kind', 'etag', 'selfLink', 'webContentLink', 'alternateLink', 'embedLink', 'iconLink', 'title', 'mimeType', 'labels', 'copyRequiresWriterPermission', 'createdDate', 'modifiedDate', 'modifiedByMeDate', 'lastViewedByMeDate', 'markedViewedByMeDate', 'version', 'parents', 'downloadUrl', 'userPermission', 'originalFilename', 'fileExtension', 'md5Checksum', 'fileSize', 'quotaBytesUsed', 'ownerNames', 'owners', 'lastModifyingUserName', 'lastModifyingUser', 'capabilities', 'editable', 'copyable', 'writersCanShare', 'shared', 'explicitlyTrashed', 'appDataContents', 'headRevisionId', 'spaces'])

In [0]:
!unzip dataset.zip

In [0]:
PATH = '/content/dataset'

In [0]:
os.listdir(PATH)

['test_set', 'training_set']

In [0]:
train_dir = os.path.join(PATH,
                         'training_set')
test_dir = os.path.join(PATH,
                        'test_set')

In [0]:
train_dogs_dir = os.path.join(train_dir,
                                'dogs')
train_cats_dir = os.path.join(train_dir,
                                   'cats')

test_dogs_dir = os.path.join(test_dir,
                               'dogs')
test_cats_dir = os.path.join(test_dir,
                                  'cats')

In [0]:
len(os.listdir(train_dogs_dir))

4000

In [0]:
len(os.listdir(train_cats_dir))

4000

In [0]:
len(os.listdir(test_dogs_dir))

1000

In [0]:
len(os.listdir(test_cats_dir))

1000

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
#Building the CNN
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3,3), input_shape = (64,64,3),activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [46]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [47]:
#Training the neural network
history = classifier.fit_generator(
    training_set,
    steps_per_epoch=8000 // 32,
    epochs=25,
    validation_data=test_set,
    validation_steps=2000 // 32,
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=4
    )]
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
Epoch 1/25
250/250 [==============================] - 75s 300ms/step - loss: 0.6902 - accuracy: 0.5624 - val_loss: 0.6488 - val_accuracy: 0.6406
Epoch 2/25
250/250 [==============================] - 74s 297ms/step - loss: 0.6427 - accuracy: 0.6400 - val_loss: 0.6108 - val_accuracy: 0.6789
Epoch 3/25
250/250 [==============================] - 74s 297ms/step - loss: 0.5882 - accuracy: 0.6865 - val_loss: 0.6038 - val_accuracy: 0.6865
Epoch 4/25
250/250 [==============================] - 74s 296ms/step - loss: 0.5410 - accuracy: 0.7306 - val_loss: 0.5140 - val_accuracy: 0.7535
Epoch 5/25
250/250 [==============================] - 74s 298ms/step - loss: 0.5103 - accuracy: 0.7430 - val_loss: 0.5314 - val_accuracy: 0.7586
Epoch 6/25
250/250 [==============================] - 74s 297ms/step - loss: 0.4767 - accuracy: 0.7674 - val_loss: 0.5456 - val_accuracy: 0.7470
Epoch 7/25
250/250 [=================

In [48]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['accuracy'],
                         mode='lines+markers',
                         name='Training accuracy'))
fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_accuracy'],
                         mode='lines+markers',
                         name='Validation accuracy'))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))
fig.show()

In [0]:
from tensorflow.keras.models import load_model
classifier.save('model.h5')
model_file = drive.CreateFile({'title' : 'model.h5'})  
model_file.SetContentFile('model.h5')
model_file.Upload()
# download to google drive                       
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '12PE_S_BjiMd6Uc151hKw2IUzSv6oPYJU'})